## --------------------- Test >>  src.download_yf 

In [120]:
# 2) importer le module et le recharger pour être sûr d'avoir la dernière version
import src.download_yf as dlyf
importlib.reload(dlyf)

# 3) appeler la fonction
tidy_all = dlyf.update_fchi_tidy()
tidy_all[tidy_all["ticker"] == "^FCHI"].tail()


1 Failed download:
['^FCHI']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-11-22 -> 2025-11-24)')


[INFO] Tidy existant chargé : /Users/souhail/projets-ml/indexpulse/data/raw/ohlcv_10y.parquet
[INFO] Dernière date FCHI dans le parquet tidy : 2025-11-21 00:00:00
[INFO] Téléchargement FCHI du 2025-11-22 au 2025-11-24…
[WARN] yf.download a renvoyé vide pour ['^FCHI'] entre 2025-11-22 et 2025-11-24
[WARN] Aucunes nouvelles lignes reçues pour FCHI.


,date,ticker,open,high,low,close,adj_close,volume
2556,2025-11-17,^FCHI,8155.930176,8176.100098,8109.819824,8119.020020,8119.020020,42960200.0
2557,2025-11-18,^FCHI,8017.729980,8032.509766,7925.060059,7967.930176,7967.930176,68564200.0
2558,2025-11-19,^FCHI,7964.770020,8005.750000,7919.359863,7953.770020,7953.770020,55042800.0
2559,2025-11-20,^FCHI,8044.850098,8057.700195,7981.069824,7981.069824,7981.069824,60357000.0
2560,2025-11-21,^FCHI,7881.100098,7996.459961,7875.870117,7982.649902,7982.649902,64784800.0


## -------------------------- Test >>  src.preprocessing.py 

In [121]:
import src.preprocessing as prep
importlib.reload(prep)

feat_fchi = prep.preprocess_cac40_from_tidy(tidy_all)
feat_fchi.tail(), feat_fchi.shape

[OK] Features FCHI écrites dans : /Users/souhail/projets-ml/indexpulse/data/processed/FCHI_features_h1d.parquet


(           date ticker         open         high          low        close  \
 2355 2025-11-14  ^FCHI  8186.140137  8202.480469  8084.229980  8170.089844   
 2356 2025-11-17  ^FCHI  8155.930176  8176.100098  8109.819824  8119.020020   
 2357 2025-11-18  ^FCHI  8017.729980  8032.509766  7925.060059  7967.930176   
 2358 2025-11-19  ^FCHI  7964.770020  8005.750000  7919.359863  7953.770020   
 2359 2025-11-20  ^FCHI  8044.850098  8057.700195  7981.069824  7981.069824   
 
         adj_close      volume  dow  month  ...       ema200  volstd200  \
 2355  8170.089844  54654300.0    4     11  ...  7840.150927   0.010450   
 2356  8119.020020  42960200.0    0     11  ...  7842.925744   0.010459   
 2357  7967.930176  68564200.0    1     11  ...  7844.169569   0.010492   
 2358  7953.770020  55042800.0    2     11  ...  7845.260121   0.010488   
 2359  7981.069824  60357000.0    3     11  ...  7846.611461   0.010487   
 
       vol_sma200    vol_lag1    vol_lag2    vol_lag5   vol_lag10       

In [123]:
import importlib, src.arima_train as arima_train
importlib.reload(arima_train)

metrics = arima_train.train_arima_logret1()
metrics

Split temporel : train=1653, val=354, test=353, n_dates=2360
Séries : train=1653, val=354, test=353
ARIMA VAL RMSE=0.008252   MAE=0.006257
ARIMA TEST RMSE=0.009796   MAE=0.007219
Modèle ARIMA sauvegardé sous : /Users/souhail/projets-ml/indexpulse/models/arima_logret1_FCHI.pkl


/Users/souhail/projets-ml/indexpulse/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/souhail/projets-ml/indexpulse/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/souhail/projets-ml/indexpulse/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/souhail/projets-ml/indexpulse/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is availab

{'rmse_val': 0.008251699693177503,
 'mae_val': 0.006257279383915757,
 'rmse_test': 0.009796097597593959,
 'mae_test': 0.007218910492449248,
 'model_path': PosixPath('/Users/souhail/projets-ml/indexpulse/models/arima_logret1_FCHI.pkl')}

In [ ]:
from pathlib import Path
import os
import numpy as np
import pandas as pd
import yfinance as yf

TICKERS = ["^GSPC", "^FCHI", "^STOXX50E"]

end = pd.Timestamp.today().normalize()
start = end - pd.DateOffset(years=10)

start, end


(Timestamp('2015-11-21 00:00:00'), Timestamp('2025-11-21 00:00:00'))

## 1) Téléchargements : Appel multi-tickers 

In [60]:
raw = yf.download(
    TICKERS,
    start=start.date().isoformat(),
    end=end.date().isoformat(),
    interval="1d",
    group_by="ticker",
    auto_adjust=False,   # on veut les 4 prix bruts + Adj Close séparé
    actions=False,
    progress=False,
    threads=True
)
raw.tail()


Ticker            ^GSPC                                                      \
Price              Open         High          Low        Close    Adj Close   
Date                                                                          
2025-11-14  6672.140137  6774.310059  6646.870117  6734.109863  6734.109863   
2025-11-17  6713.609863  6754.500000  6638.899902  6672.410156  6672.410156   
2025-11-18  6641.189941  6666.629883  6574.319824  6617.319824  6617.319824   
2025-11-19  6625.839844  6689.750000  6603.500000  6642.160156  6642.160156   
2025-11-20  6737.930176  6770.350098  6534.049805  6538.759766  6538.759766   

Ticker                      ^STOXX50E                                         \
Price             Volume         Open         High          Low        Close   
Date                                                                           
2025-11-14  5.042660e+09  5731.160156  5731.160156  5631.180176  5693.770020   
2025-11-17  5.204220e+09  5692.129883  5701.640137  5629.339844  5640.939941   
2025-11-18  5.159390e+09  5610.750000  5610.750000  5509.100098  5534.709961   
2025-11-19  5.021610e+09  5532.910156  5584.350098  5506.100098  5542.049805   
2025-11-20  5.596080e+09  5569.470215  5625.209961  5569.470215  5569.919922   

Ticker                                     ^FCHI                            \
Price         Adj Close      Volume         Open         High          Low   
Date                                                                         
2025-11-14  5693.770020  23151100.0  8186.140137  8202.480469  8084.229980   
2025-11-17  5640.939941  16799000.0  8155.930176  8176.100098  8109.819824   
2025-11-18  5534.709961  23805300.0  8017.729980  8032.509766  7925.060059   
2025-11-19  5542.049805  19950000.0  7964.770020  8005.750000  7919.359863   
2025-11-20  5569.919922  18291000.0  8044.850098  8057.700195  7981.069824   

Ticker                                            
Price             Close    Adj Close      Volume  
Date                                              
2025-11-14  8170.089844  8170.089844  54654300.0  
2025-11-17  8119.020020  8119.020020  42960200.0  
2025-11-18  7967.930176  7967.930176  68564200.0  
2025-11-19  7953.770020  7953.770020  55042800.0  
2025-11-20  7981.069824  7981.069824  60357000.0

## 2) Format Tidy

In [61]:
# 'raw' a des colonnes multi-index (Champ x Ticker). On empile le 1er niveau (tickers).
tidy = (
    raw.stack(level=0)
       .rename_axis(index=["date", "ticker"])
       .reset_index()
       .rename(columns={
           "Open": "open",
           "High": "high",
           "Low": "low",
           "Close": "close",
           "Adj Close": "adj_close",
           "Volume": "volume"
       })
       .sort_values(["ticker","date"])
       .reset_index(drop=True)
)




/var/folders/90/0y2s7s2s13gg5ppzrhs4cb500000gn/T/ipykernel_63213/3283111897.py:3: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  raw.stack(level=0)


In [ ]:
print(tidy.shape)
print(tidy.columns)

summary = (
    tidy.groupby("ticker")
        .agg(first_date=("date","min"),
             last_date=("date","max"),
             rows=("date","size"),
             missing_volume=("volume", lambda s: int(s.isna().sum())))
)
summary


#project_root=Path(os.getcwd()).parent
#data_dir.mkdir(parents=True, exist_ok=True)
#tidy.to_parquet(data_dir/'ohlcv_10y_201125.parquet', index=False)

,first_date,last_date,rows,missing_volume
ticker,,,,
^FCHI,2015-11-23,2025-11-20,2560,0
^GSPC,2015-11-23,2025-11-20,2514,0
^STOXX50E,2015-11-23,2025-11-20,2514,0


##  Feature engineering 

In [85]:
TICKERS = ["^GSPC", "^FCHI", "^STOXX50E"]  # déjà téléchargés dans `tidy`

# Fenêtres & lags de base (ajuste si besoin)
FE_WINDOWS = [5, 10, 20, 50, 100, 200]
FE_LAGS = [1, 2, 5, 10]
HORIZON_D = 1  # on prédit le retour à J+1

#### Fonctions utilitaires 

In [86]:
def add_calendar_features(df: pd.DataFrame) -> pd.DataFrame:
    di = df["date"]
    df["dow"] = di.dt.dayofweek        # 0=Mon ... 4=Fri
    df["month"] = di.dt.month
    # .is_month_end / .is_quarter_end
    df["is_month_end"] = di.dt.is_month_end.astype(int)
    df["is_quarter_end"] = di.dt.is_quarter_end.astype(int)
    # semaine ISO et jour de l'année (optionnel)
    iso = di.dt.isocalendar()
    df["weekofyear"] = iso.week.astype(int)
    df["dayofyear"] = di.dt.dayofyear
    return df

def add_return_features(df: pd.DataFrame) -> pd.DataFrame:
    # retours basés sur CLOSE
    df["ret1"] = df["close"].pct_change()
    df["logret1"] = np.log(df["close"] / df["close"].shift(1))
    # lags de retours
    for L in FE_LAGS:
        df[f"ret_lag{L}"] = df["ret1"].shift(L)
        df[f"logret_lag{L}"] = df["logret1"].shift(L)
    return df

def add_ma_vol_features(df: pd.DataFrame) -> pd.DataFrame:
    # SMA/EMA sur CLOSE + volatilité (std des retours) + volume laggé
    for w in FE_WINDOWS:
        df[f"sma{w}"] = df["close"].rolling(w, min_periods=w).mean()
        df[f"ema{w}"] = df["close"].ewm(span=w, adjust=False, min_periods=w).mean()
        df[f"volstd{w}"] = df["ret1"].rolling(w, min_periods=w).std()
        df[f"vol_sma{w}"] = df["volume"].rolling(w, min_periods=w).mean()
    for L in FE_LAGS:
        df[f"vol_lag{L}"] = df["volume"].shift(L)
    return df

def add_atr(df: pd.DataFrame, n: int = 14) -> pd.DataFrame:
    # True Range = max(high-low, |high-prev_close|, |low-prev_close|)
    prev_close = df["close"].shift(1)
    tr = pd.concat([
        (df["high"] - df["low"]).abs(),
        (df["high"] - prev_close).abs(),
        (df["low"] - prev_close).abs()
    ], axis=1).max(axis=1)
    df["atr"] = tr.rolling(n, min_periods=n).mean()
    return df

def make_target(df: pd.DataFrame, horizon: int = 1) -> pd.DataFrame:
    # target = retour futur sur CLOSE
    df["target_ret"] = df["close"].pct_change(horizon).shift(-horizon)
    df["target_logret"] = (np.log(df["close"] / df["close"].shift(horizon))).shift(-horizon)
    return df

def build_features_one_ticker(tidy_one: pd.DataFrame, horizon: int = 1) -> pd.DataFrame:
    df = tidy_one.sort_values("date").copy()
    df = add_calendar_features(df)
    df = add_return_features(df)
    df = add_ma_vol_features(df)
    df = add_atr(df, n=14)
    df = make_target(df, horizon=horizon)

    # Nettoyage
    need_cols = ["ret1", "logret1", "atr", "target_ret"]
    need_cols += [f"sma{w}" for w in FE_WINDOWS] + [f"volstd{w}" for w in FE_WINDOWS]
    df = df.dropna(subset=need_cols).reset_index(drop=True)

    # ⚠️ Pas de standardisation ici : on garde les vraies échelles
    # (on pourra scaler dynamiquement plus tard dans le pipeline de modélisation)

    return df


In [70]:
features_by_ticker = {}
for t in TICKERS:
    one = tidy[tidy["ticker"] == t].copy()
    feat = build_features_one_ticker(one, horizon=HORIZON_D)
    features_by_ticker[t] = feat
    print(t, feat.shape, "de", one.shape)

# Exemples d’aperçu
features_by_ticker["^GSPC"].head()


^GSPC (2313, 55) de (2514, 8)
^FCHI (2359, 55) de (2560, 8)
^STOXX50E (2313, 55) de (2514, 8)


Price,date,ticker,open,high,low,close,adj_close,volume,dow,month,...,ema200,volstd200,vol_sma200,vol_lag1,vol_lag2,vol_lag5,vol_lag10,atr,target_ret,target_logret
0,2016-09-09,^GSPC,2169.080078,2169.080078,2127.810059,2127.810059,2127.810059,4.233960e+09,4,9,...,2094.749187,0.009270,3.937209e+09,3.727840e+09,3.319420e+09,3.392120e+09,2.969310e+09,16.172137,0.014677,0.014570
1,2016-09-12,^GSPC,2120.860107,2163.300049,2119.120117,2159.040039,2159.040039,4.010480e+09,0,9,...,2095.388897,0.009327,3.937836e+09,4.233960e+09,3.727840e+09,3.091120e+09,3.342340e+09,18.671422,-0.014831,-0.014942
2,2016-09-13,^GSPC,2150.469971,2150.469971,2120.270020,2127.020020,2127.020020,4.141670e+09,1,9,...,2095.703635,0.009387,3.944280e+09,4.010480e+09,4.233960e+09,3.447650e+09,2.654780e+09,20.670707,-0.000588,-0.000588
3,2016-09-14,^GSPC,2127.860107,2141.330078,2119.899902,2125.770020,2125.770020,3.664100e+09,2,9,...,2096.002803,0.009387,3.955266e+09,4.141670e+09,4.010480e+09,3.319420e+09,3.006800e+09,21.083583,0.010109,0.010059
4,2016-09-15,^GSPC,2125.360107,2151.310059,2122.360107,2147.260010,2147.260010,3.373720e+09,3,9,...,2096.512825,0.009408,3.950760e+09,3.664100e+09,4.141670e+09,3.727840e+09,3.766390e+09,22.490008,-0.003772,-0.003779


In [90]:
features_by_ticker

{'^GSPC': Price       date ticker         open         high          low        close  \
 0     2016-09-09  ^GSPC  2169.080078  2169.080078  2127.810059  2127.810059   
 1     2016-09-12  ^GSPC  2120.860107  2163.300049  2119.120117  2159.040039   
 2     2016-09-13  ^GSPC  2150.469971  2150.469971  2120.270020  2127.020020   
 3     2016-09-14  ^GSPC  2127.860107  2141.330078  2119.899902  2125.770020   
 4     2016-09-15  ^GSPC  2125.360107  2151.310059  2122.360107  2147.260010   
 ...          ...    ...          ...          ...          ...          ...   
 2308  2025-11-13  ^GSPC  6826.470215  6828.049805  6724.720215  6737.490234   
 2309  2025-11-14  ^GSPC  6672.140137  6774.310059  6646.870117  6734.109863   
 2310  2025-11-17  ^GSPC  6713.609863  6754.500000  6638.899902  6672.410156   
 2311  2025-11-18  ^GSPC  6641.189941  6666.629883  6574.319824  6617.319824   
 2312  2025-11-19  ^GSPC  6625.839844  6689.750000  6603.500000  6642.160156   
 
 Price    adj_close        vo

In [94]:
project_root=Path(os.getcwd()).parent
proc_dir=project_root/"data/processed"

for ticker, df in features_by_ticker.items():
    name = ticker.replace("^", "")  # ^FCHI -> FCHI
    path = proc_dir / f"{name}_features_h1d.parquet"
    df.to_parquet(path, index=False)
    print("Sauvé :", path)

Sauvé : /Users/souhail/projets-ml/indexpulse/data/processed/GSPC_features_h1d.parquet
Sauvé : /Users/souhail/projets-ml/indexpulse/data/processed/FCHI_features_h1d.parquet
Sauvé : /Users/souhail/projets-ml/indexpulse/data/processed/STOXX50E_features_h1d.parquet


In [97]:
df_cac  = pd.read_parquet(proc_dir/"FCHI_features_h1d.parquet")
df_cac.tail()

,date,ticker,open,high,low,close,adj_close,volume,dow,month,...,ema200,volstd200,vol_sma200,vol_lag1,vol_lag2,vol_lag5,vol_lag10,atr,target_ret,target_logret
2354,2025-11-13,^FCHI,8270.849609,8314.230469,8227.879883,8232.490234,8232.490234,54230000.0,3,11,...,7836.834958,0.010444,67756191.0,61916100.0,47538300.0,62779100.0,74718500.0,88.552909,-0.007580,-0.007609
2355,2025-11-14,^FCHI,8186.140137,8202.480469,8084.229980,8170.089844,8170.089844,54654300.0,4,11,...,7840.150927,0.010450,67610487.5,54230000.0,61916100.0,52404200.0,60682000.0,95.577218,-0.006251,-0.006270
2356,2025-11-17,^FCHI,8155.930176,8176.100098,8109.819824,8119.020020,8119.020020,42960200.0,0,11,...,7842.925744,0.010459,67505916.5,54654300.0,54230000.0,52949300.0,47263500.0,97.480818,-0.018609,-0.018785
2357,2025-11-18,^FCHI,8017.729980,8032.509766,7925.060059,7967.930176,7967.930176,68564200.0,1,11,...,7844.169569,0.010492,67327108.0,42960200.0,54654300.0,47538300.0,54448200.0,108.455776,-0.001777,-0.001779
2358,2025-11-19,^FCHI,7964.770020,8005.750000,7919.359863,7953.770020,7953.770020,55042800.0,2,11,...,7845.260121,0.010488,67167644.5,68564200.0,42960200.0,61916100.0,49985700.0,108.047956,0.003432,0.003426


In [ ]:
df = df_cac.copy()
df = df.sort_values("date")


# dates uniques triées
unique_dates = df["date"].sort_values().unique()
n_dates = len(unique_dates)
train_end = unique_dates[int(0.7 * n_dates)]   # ~70% des dates
val_end   = unique_dates[int(0.85 * n_dates)]  # ~15% val, ~15% test

train = df[df["date"] <= train_end]
val   = df[(df["date"] > train_end) & (df["date"] <= val_end)]
test  = df[df["date"] > val_end]

len(train), len(val), len(test), n_dates

(1652, 354, 353, 2359)

In [76]:
feature_cols = [
    c for c in df.columns
    if c not in ("date", "ticker", "target_ret", "target_logret")
]

X_train, y_train = train[feature_cols], train["target_logret"]
X_val,   y_val   = val[feature_cols],   val["target_logret"]
X_test,  y_test  = test[feature_cols],  test["target_logret"]

X_train.shape, X_val.shape, X_test.shape


((1652, 51), (354, 51), (353, 51))

In [77]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

def eval_metrics(y_true, y_pred, name=""):
    rmse = float(np.sqrt(mean_squared_error(y_true, y_pred)))
    mae  = float(mean_absolute_error(y_true, y_pred))
    print(f"{name} RMSE={rmse:.6f}  MAE={mae:.6f}")
    return rmse, mae

model_xgb = XGBRegressor(
    n_estimators=500,
    max_depth=4,
    learning_rate=0.03,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="reg:squarederror",
    random_state=42,
)


model_xgb.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=False,
)

model_xgb.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=False,
)

pred_val  = model_xgb.predict(X_val)
pred_test = model_xgb.predict(X_test)

rmse_val, mae_val   = eval_metrics(y_val,  pred_val,  "XGB VAL")
rmse_test, mae_test = eval_metrics(y_test, pred_test, "XGB TEST")


XGB VAL RMSE=0.022900  MAE=0.020633
XGB TEST RMSE=0.023496  MAE=0.021661


In [78]:
from statsmodels.tsa.arima.model import ARIMA

# série de logret1 indexée par date
series = df.set_index("date")["logret1"].sort_index()

train_s = series[series.index <= train_end].dropna()
val_s   = series[(series.index > train_end) & (series.index <= val_end)].dropna()
test_s  = series[series.index > val_end].dropna()

len(train_s), len(val_s), len(test_s)




(1652, 354, 353)

In [79]:
# ARIMA sur train, prévision sur val
arima_model = ARIMA(train_s, order=(1, 0, 1))
arima_res   = arima_model.fit()

fc_val = arima_res.forecast(steps=len(val_s))
fc_val.index = val_s.index  # aligner les index

rmse_val_arima, mae_val_arima = eval_metrics(val_s, fc_val, "ARIMA VAL")

# Refit sur train+val, prévision sur test
trainval_s = series[series.index <= val_end].dropna()
arima_model_tv = ARIMA(trainval_s, order=(1,0,1))
arima_res_tv   = arima_model_tv.fit()

fc_test = arima_res_tv.forecast(steps=len(test_s))
fc_test.index = test_s.index

rmse_test_arima, mae_test_arima = eval_metrics(test_s, fc_test, "ARIMA TEST")


/Users/souhail/projets-ml/indexpulse/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/souhail/projets-ml/indexpulse/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/souhail/projets-ml/indexpulse/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/souhail/projets-ml/indexpulse/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is availab

ARIMA VAL RMSE=0.008250  MAE=0.006251
ARIMA TEST RMSE=0.009796  MAE=0.007218


/Users/souhail/projets-ml/indexpulse/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/souhail/projets-ml/indexpulse/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [80]:
from prophet import Prophet

df_prophet = (
    df[["date", "logret1"]]
    .rename(columns={"date": "ds", "logret1": "y"})
    .dropna()
)

train_p = df_prophet[df_prophet["ds"] <= train_end]
val_p   = df_prophet[(df_prophet["ds"] > train_end) & (df_prophet["ds"] <= val_end)]
test_p  = df_prophet[df_prophet["ds"] > val_end]

m = Prophet(
    weekly_seasonality=True,
    yearly_seasonality=True,
    daily_seasonality=False,
    seasonality_mode="additive",
)
m.fit(train_p)

# Prévisions sur la validation
fc_val_p = m.predict(val_p[["ds"]])
pred_val_p = fc_val_p["yhat"].values

rmse_val_prophet, mae_val_prophet = eval_metrics(val_p["y"], pred_val_p, "Prophet VAL")

# Refit sur train+val, prévisions sur test
trainval_p = df_prophet[df_prophet["ds"] <= val_end]
m_tv = Prophet(
    weekly_seasonality=True,
    yearly_seasonality=True,
    daily_seasonality=False,
    seasonality_mode="additive",
)
m_tv.fit(trainval_p)

fc_test_p = m_tv.predict(test_p[["ds"]])
pred_test_p = fc_test_p["yhat"].values

rmse_test_prophet, mae_test_prophet = eval_metrics(test_p["y"], pred_test_p, "Prophet TEST")


14:50:11 - cmdstanpy - INFO - Chain [1] start processing
14:50:11 - cmdstanpy - INFO - Chain [1] done processing
14:50:11 - cmdstanpy - INFO - Chain [1] start processing
14:50:11 - cmdstanpy - INFO - Chain [1] done processing


Prophet VAL RMSE=0.008473  MAE=0.006468
Prophet TEST RMSE=0.010001  MAE=0.007399


In [81]:
print(df["logret1"].std())
print(df["logret1"].mean())


0.011143706238255854
0.00023749096156245697


### Sauvegarde modèle ARIMA 

In [82]:
from pathlib import Path

project_root = Path.cwd().parent          # ../ depuis notebooks/
models_dir = project_root / "models"
models_dir.mkdir(exist_ok=True)

arima_path = models_dir / "arima_logret1_FCHI.pkl"
arima_res_tv.save(arima_path)

print("ARIMA sauvegardé sous", arima_path)



ARIMA sauvegardé sous /Users/souhail/projets-ml/indexpulse/models/arima_logret1_FCHI.pkl
